In [97]:
import pulp
import numpy as np
import matplotlib_inline as plt

In [98]:
import numpy as np

def generate_random_directed_graph(num_nodes, num_edges, cost_range=(1, 10)):
    if num_edges > num_nodes * (num_nodes - 1) // 2:
        raise ValueError("Number of edges cannot exceed num_nodes * (num_nodes - 1) // 2 in a directed graph without mutual connections.")

    nodes = [chr(i) for i in range(97, 97 + num_nodes)]

    all_possible_edges = [(nodes[i], nodes[j]) for i in range(num_nodes) for j in range(i+1, num_nodes)]

    np.random.shuffle(all_possible_edges)
    directed_edges = all_possible_edges[:num_edges]

    edges_with_costs = {edge: np.random.randint(cost_range[0], cost_range[1] + 1) for edge in directed_edges}

    return nodes, edges_with_costs

num_nodes = 6  
num_edges = 7
nodes, edges_with_costs = generate_random_directed_graph(num_nodes, num_edges)

print("Nodes:", nodes)
print("Directed edges with costs:", edges_with_costs)

Nodes: ['a', 'b', 'c', 'd', 'e', 'f']
Directed edges with costs: {('e', 'f'): 6, ('d', 'e'): 6, ('b', 'f'): 8, ('b', 'e'): 9, ('a', 'd'): 3, ('c', 'f'): 8, ('a', 'e'): 3}


In [99]:
prob = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)

In [100]:

#nodes = ['a','b','c','d','r']
#edges_with_costs ={
    #('a','b'):3,('c','a'):1,('r','a'):9,('b','c'):8,
    #('b','r'):4,('d','b'):6,('r','c'):2,
    #('c','d'):7,('d','r'):5
#}


In [101]:
import pulp
x = pulp.LpVariable.dicts('x',(nodes,nodes),0,1,cat=pulp.LpInteger)

In [102]:
prob +=pulp.lpSum(edges_with_costs[u,v] * x[u][v] for u, v in edges_with_costs)

In [103]:
for v in nodes:
    if v != 'r':
        prob +=pulp.lpSum(x[u][v] for u in nodes if (u,v) in edges_with_costs) == 1
        #prob +=pulp.lpSum(x[u,v] for u in nodes if (v,u) in edges_with_costs) == 1

In [104]:
prob += pulp.lpSum(x[u]['r'] for u in nodes if (u,'r') in edges_with_costs) == 0
prob += pulp.lpSum(x['r'][v] for v in nodes if ('r',v) in edges_with_costs)>= 1

In [105]:
from pulp import *
n = len(nodes)
subsets = [tuple(c) for c in allcombinations(nodes,n-1)]
subset = []
for s in subsets:
    if 'r' in s:
        subset.append(s)
subset

[]

In [106]:
for s in subset:
    sc = list(set(nodes)-set(s)) # complement of subset s
    summation = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation += x[u][v]
    prob += summation >= 1  


In [107]:
prob.solve()
print ("Status:", LpStatus[prob.status])

Status: Infeasible


In [108]:
print ("Optimal Solution")
for i in nodes:
	for j in nodes:
		if x[i][j].value() == 1:
			print ("(%s,%s)"%(i,j))  


Optimal Solution
(a,e)
